In [ ]:
import sys
import os

# 1. 設置 Protobuf 使用 Python 實現
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

# 2. 關鍵修復：屏蔽 TensorFlow
# 這會防止 transformers 導入 tensorflow，從而避免 protobuf 版本衝突
sys.modules["tensorflow"] = None 

In [ ]:
import gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, AutoConfig
from tqdm.auto import tqdm

In [15]:
# 遍歷 input 目錄，查看檔案結構
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/deberta-v3-base-offline/spm.model
/kaggle/input/deberta-v3-base-offline/config.json
/kaggle/input/deberta-v3-base-offline/tokenizer.json
/kaggle/input/deberta-v3-base-offline/tokenizer_config.json
/kaggle/input/deberta-v3-base-offline/model.safetensors
/kaggle/input/deberta-v3-base-offline/special_tokens_map.json
/kaggle/input/deberta-v3-base-offline/added_tokens.json
/kaggle/input/google-quest-challenge/sample_submission.csv
/kaggle/input/google-quest-challenge/train.csv
/kaggle/input/google-quest-challenge/test.csv
/kaggle/input/quest-finetuned-weights/best-deberta-v3-base-ft.pth


In [16]:
# ==========================================
# 1. Configuration
# ==========================================
class Config:
    """Inference configuration"""
    model_name = "/kaggle/input/deberta-v3-base-offline"
    max_len = 512
    batch_size = 16  # Can increase batch size for inference
    num_workers = 2
    seed = 42
    
    # Paths - Updated for Kaggle environment
    train_csv = "/kaggle/input/google-quest-challenge/train.csv" 
    test_csv = "/kaggle/input/google-quest-challenge/test.csv"
    sample_submission_csv = "/kaggle/input/google-quest-challenge/sample_submission.csv"
    
    # Path to the trained model weights
    model_path = "/kaggle/input/quest-finetuned-weights/best-deberta-v3-base-ft.pth"
    
    target_cols = [
        'question_asker_intent_understanding', 'question_body_critical', 'question_conversational',
        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent',
        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
        'question_type_compare', 'question_type_consequence', 'question_type_definition',
        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written',
        'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
        'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure',
        'answer_type_reason_explanation', 'answer_well_written'
    ]

def seed_everything(seed=42):
    """Set random seeds for reproducibility"""
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(Config.seed)

# Display configuration
print("✅ Configuration loaded successfully!\n")
print(f"🤖 Model: {Config.model_name}")
print(f"📏 Max Length: {Config.max_len}")
print(f"📦 Batch Size: {Config.batch_size}")
print(f"🎲 Random Seed: {Config.seed}")
print(f"\n📊 Number of target labels: {len(Config.target_cols)}")


✅ Configuration loaded successfully!

🤖 Model: /kaggle/input/deberta-v3-base-offline
📏 Max Length: 512
📦 Batch Size: 16
🎲 Random Seed: 42

📊 Number of target labels: 30


In [17]:
# ==========================================
# 2. Dataset Class
# ==========================================
class QuestDataset(Dataset):
    """Custom dataset for Q&A labeling task"""
    
    def __init__(self, df, tokenizer, max_len=512, mode="test"):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.mode = mode
        
        self.titles = df['question_title'].values
        self.bodies = df['question_body'].values
        self.answers = df['answer'].values
        
        if self.mode != "test":
            self.targets = df[Config.target_cols].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        title = str(self.titles[idx])
        body = str(self.bodies[idx])
        answer = str(self.answers[idx])
        
        # Combine question parts
        q_text = title + " " + self.tokenizer.sep_token + " " + body
        a_text = answer
        
        # Tokenize
        q_tokens = self.tokenizer.tokenize(q_text)
        a_tokens = self.tokenizer.tokenize(a_text)
        
        # Dynamic truncation with budget awareness
        budget = self.max_len - 3  # [CLS], [SEP], [SEP]
        if len(q_tokens) + len(a_tokens) > budget:
            half = budget // 2
            if len(a_tokens) > half and len(q_tokens) > half:
                a_tokens = a_tokens[:half]
                q_tokens = q_tokens[:budget - len(a_tokens)]
            elif len(a_tokens) <= half:
                q_tokens = q_tokens[:budget - len(a_tokens)]
            else:
                a_tokens = a_tokens[:budget - len(q_tokens)]
                
        # Build input IDs
        ids = [self.tokenizer.cls_token_id] + \
              self.tokenizer.convert_tokens_to_ids(q_tokens) + \
              [self.tokenizer.sep_token_id] + \
              self.tokenizer.convert_tokens_to_ids(a_tokens) + \
              [self.tokenizer.sep_token_id]
              
        mask = [1] * len(ids)
        padding_len = self.max_len - len(ids)
        ids = ids + [self.tokenizer.pad_token_id] * padding_len
        mask = mask + [0] * padding_len
        
        output = {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long)
        }
        
        if self.mode != "test":
            output['labels'] = torch.tensor(self.targets[idx], dtype=torch.float)
            
        return output

In [18]:
# ==========================================
# 3. Model Class
# ==========================================
class QuestDebertaModel(nn.Module):
    """DeBERTa model with weighted layer pooling and multi-sample dropout"""
    
    def __init__(self, model_name=Config.model_name, num_labels=30):
        super().__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.config.output_hidden_states = True
        self.model = AutoModel.from_pretrained(model_name, config=self.config)
        
        # Weighted layer pooling
        n_weights = self.config.num_hidden_layers + 1
        weights_init = torch.zeros(n_weights).float()
        weights_init.data[:-1] = -3
        self.layer_weights = nn.Parameter(weights_init)
        
        # Multi-sample dropout
        self.dropouts = nn.ModuleList([nn.Dropout(0.5) for _ in range(5)])
        self.fc = nn.Linear(self.config.hidden_size, num_labels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.hidden_states 
        
        # Stack [CLS] tokens
        cls_outputs = torch.stack([layer[:, 0, :] for layer in hidden_states], dim=1)
        
        # Weighted sum
        weights = torch.softmax(self.layer_weights, dim=0).view(1, -1, 1)
        weighted_cls = (weights * cls_outputs).sum(dim=1)
        
        # Multi-sample dropout
        logits_list = []
        for dropout in self.dropouts:
            logits_list.append(self.fc(dropout(weighted_cls)))
        avg_logits = torch.mean(torch.stack(logits_list, dim=0), dim=0)
        
        return self.sigmoid(avg_logits)

In [19]:
# ==========================================
# 4. Post-processing Utilities
# ==========================================
def get_valid_bins(train_df, target_cols):
    """Extract valid bin values from training data"""
    unique_bins = {}
    for col in target_cols:
        bins = np.sort(train_df[col].unique())
        unique_bins[col] = bins
    return unique_bins

def snap_predictions(predictions, target_cols, unique_bins):
    """Round predictions to nearest valid values from training set"""
    snapped = predictions.copy()
    for i, col in enumerate(target_cols):
        valid_vals = unique_bins[col]
        
        # Only snap sparse columns (< 100 unique values)
        if len(valid_vals) > 100:
            continue
            
        col_preds = snapped[:, i].reshape(-1, 1)
        diffs = np.abs(col_preds - valid_vals.reshape(1, -1))
        min_indices = np.argmin(diffs, axis=1)
        snapped[:, i] = valid_vals[min_indices]
    return snapped

In [ ]:
# ==========================================
# 5. Inference Pipeline
# ==========================================
@torch.no_grad()
def generate_predictions(model, test_loader, device):
    """Generate predictions on test set"""
    model.eval()
    all_preds = []
    
    print("Generating predictions...")
    for batch in tqdm(test_loader, desc="Inference"):
        input_ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        
        outputs = model(input_ids, mask)
        all_preds.append(outputs.cpu().numpy())
            
    return np.concatenate(all_preds)

def inference_pipeline():
    """Complete inference pipeline"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}\n")
    
    # Load test data
    print("Loading test data...")
    if not os.path.exists(Config.test_csv):
        print(f"Error: Test file not found at {Config.test_csv}")
        return None
        
    test_df = pd.read_csv(Config.test_csv)
    print(f"Test samples: {len(test_df)}\n")
    
    # Load tokenizer and model
    print("Loading model and tokenizer...")
    try:
        tokenizer = AutoTokenizer.from_pretrained(Config.model_name)
        model = QuestDebertaModel()
        
        if os.path.exists(Config.model_path):
            state_dict = torch.load(Config.model_path, map_location=device)
            model.load_state_dict(state_dict)
            print(f"✓ Loaded weights from {Config.model_path}")
        else:
            print(f"⚠ Model weights not found at {Config.model_path}. Using random initialization (for debugging only).")
            
        model.to(device)
    except Exception as e:
        print(f"Error loading model: {e}")
        return None
        
    print("✓ Model and tokenizer ready\n")
    
    # Prepare test dataloader
    test_dataset = QuestDataset(test_df, tokenizer, mode="test")
    test_loader = DataLoader(
        test_dataset, 
        batch_size=Config.batch_size, 
        shuffle=False, 
        num_workers=Config.num_workers
    )
    
    # Generate predictions
    final_preds = generate_predictions(model, test_loader, device)
    
    # Post-processing
    print("\nApplying post-processing...")
    if os.path.exists(Config.train_csv):
        train_df = pd.read_csv(Config.train_csv)
        bins_dict = get_valid_bins(train_df, Config.target_cols)
        final_preds = snap_predictions(final_preds, Config.target_cols, bins_dict)
        print("✓ Predictions snapped to valid values")

        epsilon = 1e-6
        noise = np.random.uniform(-epsilon, epsilon, final_preds.shape)
        final_preds = final_preds + noise
        final_preds = np.clip(final_preds, 0.0, 1.0)
        print("✓ Added epsilon noise to prevent constant columns")
    else:
        print("⚠ train.csv not found, skipping post-processing")

    # Create submission
    print("\nCreating submission file...")
    submission = pd.DataFrame(final_preds, columns=Config.target_cols)
    submission['qa_id'] = test_df['qa_id'].values
    submission = submission[['qa_id'] + Config.target_cols]
    submission.to_csv("submission.csv", index=False)
    print(f"✓ Submission saved to submission.csv")
            
    return submission

In [21]:
# ==========================================
# 7. Run Inference
# ==========================================
submission = inference_pipeline()


Using device: cuda

Loading test data...
Test samples: 476

Loading model and tokenizer...
✓ Loaded weights from /kaggle/input/quest-finetuned-weights/best-deberta-v3-base-ft.pth
✓ Model and tokenizer ready

Generating predictions...


Inference:   0%|          | 0/30 [00:00<?, ?it/s]


Applying post-processing...
✓ Predictions snapped to valid values

Creating submission file...
✓ Submission saved to submission.csv
